# Setup

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [74]:
procs_added = pd.read_csv("procs_added.txt", index_col=None, names=["tick", "machineID", "procType", "sla", "actualComp", "migrated"])
procs_current = pd.read_csv("procs_current.txt", index_col=None, names=["tick", "machineID", "isActive", "sla", "actualComp", "compDone"])
procs_done = pd.read_csv("procs_done.txt", index_col=None, names=["tick", "machineID", "procType", "sla", "ticksPassed", "actualComp"])
procs_killed = pd.read_csv("procs_killed.txt", index_col=None, names=["tick", "machineID", "sla", "compDone", "memUsed"])

util_metrics = pd.read_csv("usage.txt", index_col=None, names=["tick", "machineID", "maxTicksPassedToSlaRatio", "memUsage", "qlen", "ticksInQ", "ticksLeftOver"])

# prepare
procs_current["compLeft"] = procs_current["sla"] - procs_current["compDone"]
procs_added["compLeft"] = procs_added["sla"]
all_procs = pd.concat([procs_current[['tick', "compLeft"]], procs_added[["tick", "compLeft"]]])
load_num_procs_per_tick = procs_added[["tick"]].groupby("tick").size().reset_index(name='numProcsCurrent')
load_work_per_tick = procs_added.groupby("tick").sum().reset_index()

procs_done['timePassedAsPct'] = (100 * procs_done["ticksPassed"]) / procs_done["sla"]

procs_late = procs_done.where(procs_done["ticksPassed"] > procs_done["sla"]).dropna().reset_index(drop=True)
procs_late = procs_late.where(procs_late["ticksPassed"] > procs_late["actualComp"]).dropna().reset_index(drop=True)

proc_timings = pd.merge(procs_done, load_num_procs_per_tick, on='tick', how='left')

ticks_left = util_metrics.groupby("tick")["ticksLeftOver"].agg(['min', 'max']).reset_index()
ticks_left["range"] = ticks_left["max"] - ticks_left["min"]


In [85]:
# ticks left over while proc becoming late

procs_late["tickStarted"] = procs_late["tick"] - np.floor(procs_late["ticksPassed"])
late_procs_running = procs_late[["tickStarted", "tick"]]

late_procs_running


,tick,tickStarted
0,4.0,2.0
1,5.0,2.0
2,5.0,2.0
3,6.0,1.0
4,6.0,3.0
...,...,...
1465,999.0,995.0
1466,999.0,997.0
1467,999.0,993.0
1468,999.0,997.0


In [67]:
# late = proc_timings.where((proc_timings["timePassedAsPct"] > 100.1)).dropna()
# typed = late.where(late["procType"].str.contains("dynamic")).dropna()

# typed.sort_values("timePassedAsPct", ascending=False).reset_index(drop=True)

In [68]:
# ticks_left.where(ticks_left["range"] > 0).dropna().sort_values("range", ascending=False).reset_index(drop=True)